In [ ]:
import dotenv
from src.get_conn import get_connection_uri
dotenv.load_dotenv()

db_uri = get_connection_uri()

In [ ]:
import os
import zipfile
import psycopg2
import pandas as pd

# File paths
FILES = {
    "games": "datasets/games.csv",
    "players": "datasets/players.csv",
    "plays": "datasets/plays.csv",
    "week_data": "datasets/week_data.csv"  # This may be inside a zip
}

# Table creation SQL
TABLE_SCHEMAS = {
    "games": """
        CREATE TABLE IF NOT EXISTS games (
            gameId VARCHAR PRIMARY KEY,
            gameDate DATE,
            gameTimeEastern TIME,
            homeTeamAbbr VARCHAR,
            visitorTeamAbbr VARCHAR,
            week INTEGER
        );
    """,
    "players": """
        CREATE TABLE IF NOT EXISTS players (
            nflId VARCHAR PRIMARY KEY,
            height VARCHAR,
            weight INTEGER,
            birthDate DATE,
            collegeName VARCHAR,
            position VARCHAR,
            displayName VARCHAR
        );
    """,
    "plays": """
        CREATE TABLE IF NOT EXISTS plays (
            gameId VARCHAR,
            playId BIGINT,
            playDescription TEXT,
            quarter INTEGER,
            down INTEGER,
            yardsToGo INTEGER,
            possessionTeam VARCHAR,
            playType VARCHAR,
            yardlineSide VARCHAR,
            yardlineNumber BIGINT,
            offenseFormation VARCHAR,
            personnelO VARCHAR,
            defendersInTheBox BIGINT,
            numberOfPassRushers BIGINT,
            personnelD VARCHAR,
            typeDropback VARCHAR,
            preSnapVisitorScore BIGINT,
            preSnapHomeScore BIGINT,
            gameClock TIME,
            absoluteYardlineNumber BIGINT,
            penaltyCodes VARCHAR,
            penaltyJerseyNumbers VARCHAR,
            passResult VARCHAR,
            offensePlayResult BIGINT,
            playResult BIGINT,
            epa FLOAT,
            isDefensivePI BOOLEAN
        );
    """,
    "week_data": """
        CREATE TABLE IF NOT EXISTS week_data (
            time TIMESTAMP,
            x FLOAT,
            y FLOAT,
            s FLOAT,
            a FLOAT,
            dis FLOAT,
            o FLOAT,
            dir FLOAT,
            event VARCHAR,
            nflId VARCHAR,
            displayName VARCHAR,
            jerseyNumber INTEGER,
            position VARCHAR,
            frameId INTEGER,
            team VARCHAR,
            gameId VARCHAR,
            playId INTEGER,
            playDirection VARCHAR,
            route VARCHAR
        );
    """
}

def read_week_data():
    # If week_data.csv exists, read it directly
    if os.path.exists("datasets/week_data.csv"):
        return pd.read_csv("datasets/week_data.csv")
    # Otherwise, look for week_data.zip and extract/read the CSV
    elif os.path.exists("datasets/week_data.zip"):
        with zipfile.ZipFile("datasets/week_data.zip", "r") as z:
            with z.open("datasets/week_data.csv") as f:
                return pd.read_csv(f)
    else:
        raise FileNotFoundError("Neither week_data.csv nor week_data.zip found.")

def main():
    try:
        conn = psycopg2.connect(db_uri)
        cur = conn.cursor()

        # Create tables
        for table, schema in TABLE_SCHEMAS.items():
            cur.execute(schema)
            print(f"Created table: {table}")

        conn.commit()

        # Load and insert data
        for table, file_path in FILES.items():
            if table == "week_data":
                df = read_week_data()
            else:
                df = pd.read_csv(file_path)
            df.columns = [col.strip() for col in df.columns]  # Clean column names

            # Ensure BIGINT columns are within range and correct dtype
            BIGINT_RANGE = (-9223372036854775808, 9223372036854775807)
            # Find BIGINT columns for this table
            import re
            schema = TABLE_SCHEMAS[table]
            bigint_cols = [re.findall(r'(\w+)\s+BIGINT', line)[0]
                        for line in schema.splitlines() if 'BIGINT' in line]
            for col in bigint_cols:
                if col in df.columns:
                    # Convert to numeric, coerce errors, clip to BIGINT range, and convert to int64
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                    df[col] = df[col].clip(lower=BIGINT_RANGE[0], upper=BIGINT_RANGE[1])
                    df[col] = df[col].astype('Int64')  # Use pandas nullable integer type

            # Replace pd.NA and np.nan with None for all columns
            df = df.astype(object).where(pd.notnull(df), None)

            # Bulk insert data
            placeholders = ', '.join(['%s'] * len(df.columns))
            columns = ', '.join(df.columns)
            sql = f"INSERT INTO {table} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"
            cur.executemany(sql, df.values.tolist())

            print(f"Inserted data into: {table}")
            conn.commit()

        cur.close()
        conn.close()
        print("✅ All tables created and data inserted successfully.")

    except Exception as e:
        print("❌ Error:", e)

if __name__ == "__main__":
    main()